In [72]:
import os
import json
import pandas as pd
from fractions import Fraction

import re

In [2]:
DOWNLOAD_DATA_FILEPATH = os.getcwd() + '/data/'

In [7]:
def read_data_from_file(filepath):
    
    with open(filepath, 'r') as f:
        data_dict = json.load(f)
    
    return data_dict

In [ ]:
data_list = []

for filename in os.listdir(DOWNLOAD_DATA_FILEPATH):
    if '.json' in filename:
        data = read_data_from_file(DOWNLOAD_DATA_FILEPATH+filename)
        drinks_list = data['drinks']
        if drinks_list is not None:
            for drink in drinks_list:
                data_list.append(drink)

In [15]:
drinks_df = pd.DataFrame(data_list)
drinks_df.sample(3)

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
112,14446,Kool-Aid Slammer,None,None,None,Shot,None,Alcoholic,Shot glass,Fill half the shot glass with the kool-aid fir...,...,None,None,None,None,None,None,None,None,No,2017-09-06 23:04:33
83,11658,Loch Lomond,None,None,None,Ordinary Drink,None,Alcoholic,Cocktail glass,"In a mixing glass half-filled with ice cubes, ...",...,None,None,None,None,None,None,None,None,No,2016-07-19 11:24:41
225,11542,Jack Rose Cocktail,None,Beach,None,Ordinary Drink,None,Alcoholic,Cocktail glass,"Shake all ingredients with ice, strain into a ...",...,None,None,None,None,None,None,None,None,No,2015-08-18 15:11:08


In [17]:
ingredient_cols = [col for col in drinks_df.columns.values if 'strIngredient' in col]
measure_cols = [col for col in drinks_df.columns.values if 'strMeasure' in col]

In [31]:
a = None
'b'+str(a)

'bNone'

In [34]:
target_cols = []

for ingredient_col, measure_col in zip(ingredient_cols, measure_cols):
    
    col_name = f'{ingredient_col}-{measure_col}'
    print(col_name)
    drinks_df[col_name] = drinks_df.apply(lambda x:f'{x[ingredient_col]}-{str(x[measure_col])}', axis=1)
    
    target_cols.append(col_name)

strIngredient1-strMeasure1
strIngredient2-strMeasure2
strIngredient3-strMeasure3
strIngredient4-strMeasure4
strIngredient5-strMeasure5
strIngredient6-strMeasure6
strIngredient7-strMeasure7
strIngredient8-strMeasure8
strIngredient9-strMeasure9
strIngredient10-strMeasure10
strIngredient11-strMeasure11
strIngredient12-strMeasure12
strIngredient13-strMeasure13
strIngredient14-strMeasure14
strIngredient15-strMeasure15


In [57]:
ingredients_meta_data = []

for idx, row in drinks_df[target_cols].iterrows():
    
    for col in target_cols:
        
        if 'None' not in row[col].split('-'):
            ingredients_meta_data.append({'ingredient': row[col].split('-')[0],
                                         'measure': row[col].split('-')[1].strip()})

In [58]:
ingredients_df = pd.DataFrame(ingredients_meta_data)

In [59]:
ingredients_df

,ingredient,measure
0,Scotch,1 1/2 oz
1,Wine,1 oz Green Ginger
2,Orange juice,1 oz
3,Dark Rum,1 shot
4,Campari,1 shot
...,...,...
1553,Banana liqueur,1 part
1554,Grenadine,1 part
1555,Pineapple Juice,2 parts
1556,Orange Juice,2 parts


In [51]:
all_ingredients = list(set(list(ingredients_df.ingredient.values)))
len(all_ingredients)

319

In [88]:
all_measures = list(set(list(ingredients_df.measurement_fraction_transformed.values)))
len(all_measures)

325

In [89]:
with open(os.getcwd() + '/data/sample.txt', 'w') as f:
    f.writelines('\n'.join(all_measures))

In [85]:
def transform_fraction(measurement):
    
    fraction_regex = """\d \d\/\d|\d\/\d"""
    
    fraction = re.findall(fraction_regex, measurement)
    if len(fraction) > 0:
        fraction = fraction[0]
        
        if len(fraction.split()) == 1:
            numeric_val = float(Fraction(fraction))
        elif len(fraction.split()) == 2:
            numeric_val = float(fraction.split()[0]) + float(Fraction(fraction.split()[1]))

        replace_str = str(round(numeric_val, 2))

        return re.sub(fraction_regex, replace_str, measurement)
    else:
        return measurement

In [83]:
print(transform_fraction('1/2 cup'))
print(transform_fraction('2 1/2 cups'))
print(transform_fraction('Juice of 1/4'))
print(transform_fraction('1/2 oz Muscatel'))
print(transform_fraction('3/4 L'))

0.5 cup
2.5 cups
Juice of 0.25
0.5 oz Muscatel
0.75 L


In [69]:
x='11/2'
x=float(Fraction(x))
x=(x)     
print(x)

5.5


In [86]:
ingredients_df['measurement_fraction_transformed'] = ingredients_df.apply(lambda x:transform_fraction(x['measure']), axis=1)

ingredients_df.sample(15)

,ingredient,measure,measurement_value,measurement_fraction_transformed
1446,Strawberries,1 cup,1,1 cup
1108,Cloves,2,2,2
335,Lemon,Juice of 1/2,Juice,Juice of 0.5
256,White Rum,200 ml,200,200 ml
962,Orange juice,1 gal,1,1 gal
915,Egg,6,6,6
693,Mint,6,6,6
1166,Sugar,2 tsp,2,2 tsp
200,Soy Sauce,Dash,Dash,Dash
507,Chambord raspberry liqueur,1 oz,1,1 oz


In [87]:
ingredients_df[ingredients_df['ingredient'] == 'Sambuca']

,ingredient,measure,measurement_value,measurement_fraction_transformed
645,Sambuca,1/2 oz,1/2,0.5 oz
686,Sambuca,1 oz,1,1 oz
689,Sambuca,1/4 glass,1/4,0.25 glass
1352,Sambuca,2 cl,2,2 cl
1365,Sambuca,1 1/2 oz,1,1.5 oz
1461,Sambuca,1/3 shot,1/3,0.33 shot
1541,Sambuca,1/3 oz,1/3,0.33 oz
